In [1]:
import pandas as pd 
import pickle
from textblob import TextBlob,Word

In [2]:
def load_obj(name ):
    with open( name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [3]:
def getReview(res_id):
    reviews = []
    
    rev = df[(df['restaurant_id']==res_id) & (df['useful']>0)]['review']
    for r in rev:
        reviews.append(r)
    return reviews

In [4]:
def getPrediction(id):
    rev= getReview(id)
    food_count = 0 
    serv_count = 0
    amb_count = 0
    deal_count = 0

    for review in rev:
        prediction = {}
        prediction["food"] = food_classifier.predict([review])[0]
        prediction["service"] = service_classifier.predict([review])[0]
        prediction["amb"] = amb_classifier.predict([review])[0]
        prediction["deals"] = deal_classifier.predict([review])[0]

        if(prediction["food"]==1):
            food_count += 1
        if(prediction["service"]==1):
            serv_count += 1
        if(prediction["amb"]==1):
            amb_count += 1
        if(prediction["deals"]==1):
            deal_count += 1

    #     print(prediction)

    rev_size = len(rev)
    if(rev_size> 0 ):
        food_per = food_count/rev_size*100
        serv_per = serv_count/rev_size*100
        amb_per = amb_count/rev_size*100
        deal_per = deal_count/rev_size*100
    else:
        food_per= 0 
        serv_per=0
        amb_per=0
        deal_per=0 

    # print(rev_size,food_per,serv_per,amb_per,deal_per)
    return (rev_size,food_per,serv_per,amb_per,deal_per)

In [5]:
def stemming_tokenizer(text):
    stopwords = []
    with open('files/new_sw.txt','r+') as f:
        stopword = f.readlines()
        for sw in stopword:
            sw = sw[:-1]
            stopwords.append(sw)
    f.close()
    text = text.lower()
    words = TextBlob(text).correct().words
    words = [Word(w).lemmatize("v") for w in words if not w in stopwords]
    return words

In [6]:
df = pd.read_csv('files/reviews.csv', encoding = "ISO-8859-1")

In [7]:
food_classifier= load_obj('food_classifier')
service_classifier= load_obj('service_classifier')
amb_classifier= load_obj('amb_classifier')
deal_classifier= load_obj('deal_classifier')

In [8]:
pred= pd.read_csv('prediction.csv',encoding="ISO-8859-1")

In [11]:
for id in pred.restaurant_id:
    i,a,b,c,d= getPrediction(int(id))
    pred.iloc[id-1,1]= a
    pred.iloc[id-1,2]= b 
    pred.iloc[id-1,3]= c 
    pred.iloc[id-1,4]= d 
    print(i)
    break

7


In [13]:
print(type(id))

<class 'numpy.int64'>


In [14]:
pred.to_csv('prediction.csv', index=None)